# 전체

In [10]:
import pygame
import random
import math

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    # 객체 이동 방향 설정
    def set_move(self, x, y):
        self.move_x = x                                                                                                         # x방향으로 이동할 크기
        self.move_y = y                                                                                                         # y방향으로 이동할 크기
    # 객체 이동방향대로 위치 이동
    def update(self, width, height):
        if self.x < 0 or self.x + self.sx > width:                                                                              # 객체의 x위치가 화면을 벗어날 경우
            self.move_x = -self.move_x                                                                                          # 반대방향으로 이동방향 바꾸기
        if self.y < 0 or self.y + self.sy > height:                                                                             # 객체의 y위치가 화면을 벗어날 경우
            self.move_y = -self.move_y                                                                                          # 반대방향으로 이동방향 바꾸기
        self.x += self.move_x                                                                                                   # 객체의 x위치 이동
        self.y += self.move_y                                                                                                   # 객체의 y위치 이동

# 충돌 감지
def crash(a, b):
    if (a.x-b.sx <= b.x <= a.x+a.sx):                                                                                           # x의 위치가 겹치면서
        if (a.y-b.sy <= b.y <= a.y+a.sy):                                                                                       # y의 위치가 겹칠 경우
            return True                                                                                                         # 충돌했다 판정
    return False                                                                                                                # 그렇지 않을 경우 충돌하지 않았다 판정

# 게임 실패했을 때 사용할 배경 객체
game_over = obj()                                                                                                               # 객체 생성
game_over.put_img("./이미지/게임 오버.png")                                                                                     # 이미지 추가
game_over.change_size(size[0],size[1])                                                                                          # 이미지 크기 변경

# 게임 성공했을 때 사용할 배경 객체
game_success = obj()                                                                                                            # 객체 생성
game_success.put_img("./이미지/게임 성공.png")                                                                                  # 이미지 추가
game_success.change_size(size[0],size[1])                                                                                       # 이미지 크기 변경

# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 팩맨 먹이 리스트 생성
point_list = []
for x in range(20, size[0], 60):
    for y in range(200, size[1], 60):
        point = obj()                                                                                                           # 객체 생성
        point.put_img("./이미지/점.png")                                                                                        # 이미지 추가
        point.change_size(10,10)                                                                                                # 이미지 크기 변경
        point.x = x                                                                                                             # 객체의 초기 x위치 지정
        point.y = y                                                                                                             # 객체의 초기 y위치 지정
        point_list.append(point)                                                                                                # 먹이 리스트에 추가

# 유령 리스트 생성
monster_list = []
r = 6                                                                                                                           # 객체 이동 속도 지정
image3 = ["./이미지/분홍 유령.png", "./이미지/빨강 유령.png", "./이미지/파랑 유령.png"]                                            # 유령 이미지 리스트 생성
for x in range(35,size[0],301):
    monster = obj()                                                                                                             # 객체 생성
    monster.put_img(image3[x%3])                                                                                                # 이미지 추가
    monster.change_size(50, 50)                                                                                                 # 이미지 크기 변경
    monster.x = x                                                                                                               # 객체의 초기 x위치 지정
    monster.y = 10                                                                                                              # 객체의 초기 y위치 지정
    theta = random.randint(0,360)                                                                                               # 랜덤하게 각도 지정
    monster.set_move(r*math.cos(theta), r*math.sin(theta))                                                                      # 이동속도는 r만큼 각도 방향으로 이동할 값 지정
    monster_list.append(monster)                                                                                                # 몬스터 리스트에 추가

# 필요한 변수 선언
cnt = 0                                                                                                                         # FPS 시간 계산
direct = 1                                                                                                                      # 팩맨 입 열었는지 닫았는지 여부
time = 0                                                                                                                        # 게임 시작했는지 여부
playing = 1                                                                                                                     # 게임이 끝났는지 여부
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성
image2 = ["./이미지/팩맨2 왼쪽.png","./이미지/팩맨2 오른쪽.png","./이미지/팩맨2 위쪽.png","./이미지/팩맨2 아래쪽.png"]             # 팩맨 입 닫은 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN and playing == 1:                                                                       # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경
                    direct = 1                                                                                                  # 픽맨 입 연것으로 재설정
                    time = 1                                                                                                    # 게임이 시작 된 것으로 설정

    cnt += 1                                                                                                                    # FPS 시간 증가

    # 만약 게임이 시작되지 않았다면 초기 시간 초기화
    if time == 0:                                                                                                               # 시작이 안되었을 경우
        start_ticks = pygame.time.get_ticks()                                                                                   # 시작 시간 초기화
        font = pygame.font.Font(None, 50)                                                                                       # 폰트 설정
        cnt = 0                                                                                                                 # FPS 계산 초기화

    # FPS 시간 8마다 입 다물거나 열기
    for i in range(4):                                                                                                          # i는 0에서 3까지에 대해
        if [left_go,right_go,up_go,down_go][i] == True and cnt%8 == 0:                                                          # i방향으로 이동하면서 FPS 8로 나누어 떨어지면서
            if direct == 1:                                                                                                     # 팩맨이 입을 열었을 경우
                pacman.put_img(image2[i])                                                                                       # 팩맨을 입 닫은 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 2                                                                                                      # 입을 닫은 것으로 재설정
            elif direct == 2:                                                                                                   # 팩맨이 입을 닫았을 경우
                pacman.put_img(image[i])                                                                                        # 팩맨을 입 연 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 1                                                                                                      # 입을 연 것으로 재설정

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    # 지울 먹이 리스트를 만들어 팩맨과 먹이가 충돌했을 때(먹었을 때) 리스트에 추가
    remove_point_list = []                                                                                                      # 먹은 리스트 생성
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        if crash(pacman,p):                                                                                                     # 팩맨과 충돌하면
            remove_point_list.append(p)                                                                                         # 그 먹이를 먹은 리스트에 추가

    # 지울 먹이 리스트에 있는 모든 먹이들을 제거
    for dp in remove_point_list:                                                                                                # 모든 먹은 먹이에 대해
        point_list.remove(dp)                                                                                                   # 그 먹이를 먹이 리스트에서 제거
    
    # 남은 먹이들 그리기
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        p.show()                                                                                                                # 먹이 그리기
    
    # 유령들 그리기
    for m in monster_list:                                                                                                      # 모든 몬스터에 대해
        if cnt == 88:                                                                                                           # FPS 시간이 88로 나누어 떨어질 경우
            theta = random.randint(0,360)                                                                                       # 랜덤하게 각도 지정
            m.set_move(r*math.cos(theta), r*math.sin(theta))                                                                    # 이동속도는 r만큼 각도 방향으로 이동할 값 지정
        if time == 1 and playing == 1:                                                                                          # 게임이 시작되고 끝나지 않았을 경우
            m.update(screen_width, screen_height)                                                                               # 이동 방향대로 위치 이동
        m.show()                                                                                                                # 몬스터 그리기
        if crash(pacman,m):                                                                                                     # 팩맨과 유령이 충돌했을 경우
            game_over.show()                                                                                                    # 게임 오버 이미지 그리기
            playing = 0                                                                                                         # 게임 끝난 것으로 설정
            left_go = False                                                                                                     # 이동방향 초기화
            right_go = False                                                                                                    # 이동방향 초기화
            up_go = False                                                                                                       # 이동방향 초기화
            down_go = False                                                                                                     # 이동방향 초기화
            # 게임 진행 시간 표시
            screen.blit(time_text, (400, 750))                                                                                  # 가운데에 맞추어 시간 표시
            break                                                                                                               # for문 벗어나기
    
    # 먹이를 다 먹었을 경우 설정
    if len(point_list) == 0:                                                                                                    # 먹이 리스트의 길이가 0일 경우 (모든 먹이를 먹었을 경우)
        game_success.show()                                                                                                     # 게임 성공 이미지 그리기
        playing = 0                                                                                                             # 게임 끝난 것으로 설정
        left_go = False                                                                                                         # 이동방향 초기화
        right_go = False                                                                                                        # 이동방향 초기화
        up_go = False                                                                                                           # 이동방향 초기화
        down_go = False                                                                                                         # 이동방향 초기화
        # 게임 진행 시간 표시
        screen.blit(time_text, (400, 750))                                                                                      # 가운데에 맞추어 시간 표시
    
    # 만약 게임 중이라면 초를 다시 계산하여 표시
    if playing == 1:                                                                                                            # 게임이 끝나지 않았디면
        seconds = (pygame.time.get_ticks() - start_ticks) / 1000                                                                # 초 계산
        minute = int(seconds)//60                                                                                               # 분 계산
        seconds -= minute*60                                                                                                    # 초 재설정
        if minute == 0:                                                                                                         # 0분일 경우
            time_text = font.render(f"Time : {seconds:.2f} sec", True, (255,255,255))                                           # 초만 표시하여 폰트에 맞춰 저장
        else:                                                                                                                   # a분일 경우 (a > 0)
            time_text = font.render(f"Time : {minute:} min {seconds:.2f} sec", True, (255,255,255))                             # 분, 초를 표시하여 폰트에 맞춰 저장
        screen.blit(time_text, (10, 10))                                                                                        # 왼쪽 위에 맞추어 시간 표시

    if cnt == 88: cnt = 0                                                                                                       # FPS 시간이 너무 커지는 것을 방지하기 위해 초기화
    
    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 게임 창 생성

In [11]:
import pygame

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료

    screen.fill(color)                                                                                                          # 화면 채우기

    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 팩맨 객체 생성

In [12]:
import pygame

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    
# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 팩맨 이동

In [13]:
import pygame

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    
# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 필요한 변수 선언
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN:                                                                                        # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 팩맨 입

In [14]:
import pygame

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    
# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 필요한 변수 선언
cnt = 0                                                                                                                         # FPS 시간 계산
direct = 1                                                                                                                      # 팩맨 입 열었는지 닫았는지 여부
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성
image2 = ["./이미지/팩맨2 왼쪽.png","./이미지/팩맨2 오른쪽.png","./이미지/팩맨2 위쪽.png","./이미지/팩맨2 아래쪽.png"]             # 팩맨 입 닫은 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN:                                                                                        # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경
                    direct = 1                                                                                                  # 픽맨 입 연것으로 재설정
                    
    cnt += 1                                                                                                                    # FPS 시간 증가

    # FPS 시간 8마다 입 다물거나 열기
    for i in range(4):                                                                                                          # i는 0에서 3까지에 대해
        if [left_go,right_go,up_go,down_go][i] == True and cnt%8 == 0:                                                          # i방향으로 이동하면서 FPS 8로 나누어 떨어지면서
            if direct == 1:                                                                                                     # 팩맨이 입을 열었을 경우
                pacman.put_img(image2[i])                                                                                       # 팩맨을 입 닫은 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 2                                                                                                      # 입을 닫은 것으로 재설정
            elif direct == 2:                                                                                                   # 팩맨이 입을 닫았을 경우
                pacman.put_img(image[i])                                                                                        # 팩맨을 입 연 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 1                                                                                                      # 입을 연 것으로 재설정

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 먹이 생성

In [15]:
import pygame

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    
# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 팩맨 먹이 리스트 생성
point_list = []
for i in range(20, size[0], 60):
    for j in range(200, size[1], 60):
        point = obj()                                                                                                           # 객체 생성
        point.put_img("./이미지/점.png")                                                                                        # 이미지 추가
        point.change_size(10,10)                                                                                                # 이미지 크기 변경
        point.x = i                                                                                                             # 객체의 초기 x위치 지정
        point.y = j                                                                                                             # 객체의 초기 y위치 지정
        point_list.append(point)                                                                                                # 먹이 리스트에 추가

# 필요한 변수 선언
cnt = 0                                                                                                                         # FPS 시간 계산
direct = 1                                                                                                                      # 팩맨 입 열었는지 닫았는지 여부
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성
image2 = ["./이미지/팩맨2 왼쪽.png","./이미지/팩맨2 오른쪽.png","./이미지/팩맨2 위쪽.png","./이미지/팩맨2 아래쪽.png"]             # 팩맨 입 닫은 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN:                                                                                        # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경
                    direct = 1                                                                                                  # 픽맨 입 연것으로 재설정
                    
    cnt += 1                                                                                                                    # FPS 시간 증가

    # FPS 시간 8마다 입 다물거나 열기
    for i in range(4):                                                                                                          # i는 0에서 3까지에 대해
        if [left_go,right_go,up_go,down_go][i] == True and cnt%8 == 0:                                                          # i방향으로 이동하면서 FPS 8로 나누어 떨어지면서
            if direct == 1:                                                                                                     # 팩맨이 입을 열었을 경우
                pacman.put_img(image2[i])                                                                                       # 팩맨을 입 닫은 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 2                                                                                                      # 입을 닫은 것으로 재설정
            elif direct == 2:                                                                                                   # 팩맨이 입을 닫았을 경우
                pacman.put_img(image[i])                                                                                        # 팩맨을 입 연 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 1                                                                                                      # 입을 연 것으로 재설정

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    # 남은 먹이들 그리기
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        p.show()                                                                                                                # 먹이 그리기
    
    if cnt == 88: cnt = 0                                                                                                       # FPS 시간이 너무 커지는 것을 방지하기 위해 초기화
    
    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 팩맨이 먹이 먹었는지 판단

In [16]:
import pygame

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    
# 충돌 감지
def crash(a, b):
    if (a.x-b.sx <= b.x <= a.x+a.sx):                                                                                           # x의 위치가 겹치면서
        if (a.y-b.sy <= b.y <= a.y+a.sy):                                                                                       # y의 위치가 겹칠 경우
            return True                                                                                                         # 충돌했다 판정
    return False                                                                                                                # 그렇지 않을 경우 충돌하지 않았다 판정

# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 팩맨 먹이 리스트 생성
point_list = []
for i in range(20, size[0], 60):
    for j in range(200, size[1], 60):
        point = obj()                                                                                                           # 객체 생성
        point.put_img("./이미지/점.png")                                                                                        # 이미지 추가
        point.change_size(10,10)                                                                                                # 이미지 크기 변경
        point.x = i                                                                                                             # 객체의 초기 x위치 지정
        point.y = j                                                                                                             # 객체의 초기 y위치 지정
        point_list.append(point)                                                                                                # 먹이 리스트에 추가

# 필요한 변수 선언
cnt = 0                                                                                                                         # FPS 시간 계산
direct = 1                                                                                                                      # 팩맨 입 열었는지 닫았는지 여부
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성
image2 = ["./이미지/팩맨2 왼쪽.png","./이미지/팩맨2 오른쪽.png","./이미지/팩맨2 위쪽.png","./이미지/팩맨2 아래쪽.png"]             # 팩맨 입 닫은 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN:                                                                                        # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경
                    direct = 1                                                                                                  # 픽맨 입 연것으로 재설정
                    
    cnt += 1                                                                                                                    # FPS 시간 증가

    # FPS 시간 8마다 입 다물거나 열기
    for i in range(4):                                                                                                          # i는 0에서 3까지에 대해
        if [left_go,right_go,up_go,down_go][i] == True and cnt%8 == 0:                                                          # i방향으로 이동하면서 FPS 8로 나누어 떨어지면서
            if direct == 1:                                                                                                     # 팩맨이 입을 열었을 경우
                pacman.put_img(image2[i])                                                                                       # 팩맨을 입 닫은 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 2                                                                                                      # 입을 닫은 것으로 재설정
            elif direct == 2:                                                                                                   # 팩맨이 입을 닫았을 경우
                pacman.put_img(image[i])                                                                                        # 팩맨을 입 연 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 1                                                                                                      # 입을 연 것으로 재설정

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    # 지울 먹이 리스트를 만들어 팩맨과 먹이가 충돌했을 때(먹었을 때) 리스트에 추가
    remove_point_list = []                                                                                                      # 먹은 리스트 생성
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        if crash(pacman,p):                                                                                                     # 팩맨과 충돌하면
            remove_point_list.append(p)                                                                                         # 그 먹이를 먹은 리스트에 추가

    # 지울 먹이 리스트에 있는 모든 먹이들을 제거
    for dp in remove_point_list:                                                                                                # 모든 먹은 먹이에 대해
        point_list.remove(dp)                                                                                                   # 그 먹이를 먹이 리스트에서 제거
    
    # 남은 먹이들 그리기
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        p.show()                                                                                                                # 먹이 그리기
    
    if cnt == 88: cnt = 0                                                                                                       # FPS 시간이 너무 커지는 것을 방지하기 위해 초기화
    
    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 먹이 다 먹었는지 확인

In [17]:
import pygame

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    
# 충돌 감지
def crash(a, b):
    if (a.x-b.sx <= b.x <= a.x+a.sx):                                                                                           # x의 위치가 겹치면서
        if (a.y-b.sy <= b.y <= a.y+a.sy):                                                                                       # y의 위치가 겹칠 경우
            return True                                                                                                         # 충돌했다 판정
    return False                                                                                                                # 그렇지 않을 경우 충돌하지 않았다 판정

# 게임 성공했을 때 사용할 배경 객체
game_success = obj()                                                                                                            # 객체 생성
game_success.put_img("./이미지/게임 성공.png")                                                                                  # 이미지 추가
game_success.change_size(size[0],size[1])                                                                                       # 이미지 크기 변경

# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 팩맨 먹이 리스트 생성
point_list = []
for i in range(20, size[0], 60):
    for j in range(200, size[1], 60):
        point = obj()                                                                                                           # 객체 생성
        point.put_img("./이미지/점.png")                                                                                        # 이미지 추가
        point.change_size(10,10)                                                                                                # 이미지 크기 변경
        point.x = i                                                                                                             # 객체의 초기 x위치 지정
        point.y = j                                                                                                             # 객체의 초기 y위치 지정
        point_list.append(point)                                                                                                # 먹이 리스트에 추가

# 필요한 변수 선언
cnt = 0                                                                                                                         # FPS 시간 계산
direct = 1                                                                                                                      # 팩맨 입 열었는지 닫았는지 여부
playing = 1                                                                                                                     # 게임이 끝났는지 여부
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성
image2 = ["./이미지/팩맨2 왼쪽.png","./이미지/팩맨2 오른쪽.png","./이미지/팩맨2 위쪽.png","./이미지/팩맨2 아래쪽.png"]             # 팩맨 입 닫은 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN and playing == 1:                                                                       # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경
                    direct = 1                                                                                                  # 픽맨 입 연것으로 재설정
                    
    cnt += 1                                                                                                                    # FPS 시간 증가

    # FPS 시간 8마다 입 다물거나 열기
    for i in range(4):                                                                                                          # i는 0에서 3까지에 대해
        if [left_go,right_go,up_go,down_go][i] == True and cnt%8 == 0:                                                          # i방향으로 이동하면서 FPS 8로 나누어 떨어지면서
            if direct == 1:                                                                                                     # 팩맨이 입을 열었을 경우
                pacman.put_img(image2[i])                                                                                       # 팩맨을 입 닫은 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 2                                                                                                      # 입을 닫은 것으로 재설정
            elif direct == 2:                                                                                                   # 팩맨이 입을 닫았을 경우
                pacman.put_img(image[i])                                                                                        # 팩맨을 입 연 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 1                                                                                                      # 입을 연 것으로 재설정

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    # 지울 먹이 리스트를 만들어 팩맨과 먹이가 충돌했을 때(먹었을 때) 리스트에 추가
    remove_point_list = []                                                                                                      # 먹은 리스트 생성
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        if crash(pacman,p):                                                                                                     # 팩맨과 충돌하면
            remove_point_list.append(p)                                                                                         # 그 먹이를 먹은 리스트에 추가

    # 지울 먹이 리스트에 있는 모든 먹이들을 제거
    for dp in remove_point_list:                                                                                                # 모든 먹은 먹이에 대해
        point_list.remove(dp)                                                                                                   # 그 먹이를 먹이 리스트에서 제거
    
    # 남은 먹이들 그리기
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        p.show()                                                                                                                # 먹이 그리기
    
    # 먹이를 다 먹었을 경우 설정
    if len(point_list) == 0:                                                                                                    # 먹이 리스트의 길이가 0일 경우 (모든 먹이를 먹었을 경우)
        game_success.show()                                                                                                     # 게임 성공 이미지 그리기
        playing = 0                                                                                                             # 게임 끝난 것으로 설정
        left_go = False                                                                                                         # 이동방향 초기화
        right_go = False                                                                                                        # 이동방향 초기화
        up_go = False                                                                                                           # 이동방향 초기화
        down_go = False                                                                                                         # 이동방향 초기화
        
    if cnt == 88: cnt = 0                                                                                                       # FPS 시간이 너무 커지는 것을 방지하기 위해 초기화
    
    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 시간 출력

In [18]:
import pygame

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    
# 충돌 감지
def crash(a, b):
    if (a.x-b.sx <= b.x <= a.x+a.sx):                                                                                           # x의 위치가 겹치면서
        if (a.y-b.sy <= b.y <= a.y+a.sy):                                                                                       # y의 위치가 겹칠 경우
            return True                                                                                                         # 충돌했다 판정
    return False                                                                                                                # 그렇지 않을 경우 충돌하지 않았다 판정

# 게임 성공했을 때 사용할 배경 객체
game_success = obj()                                                                                                            # 객체 생성
game_success.put_img("./이미지/게임 성공.png")                                                                                  # 이미지 추가
game_success.change_size(size[0],size[1])                                                                                       # 이미지 크기 변경

# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 팩맨 먹이 리스트 생성
point_list = []
for i in range(20, size[0], 60):
    for j in range(200, size[1], 60):
        point = obj()                                                                                                           # 객체 생성
        point.put_img("./이미지/점.png")                                                                                        # 이미지 추가
        point.change_size(10,10)                                                                                                # 이미지 크기 변경
        point.x = i                                                                                                             # 객체의 초기 x위치 지정
        point.y = j                                                                                                             # 객체의 초기 y위치 지정
        point_list.append(point)                                                                                                # 먹이 리스트에 추가

# 필요한 변수 선언
cnt = 0                                                                                                                         # FPS 시간 계산
direct = 1                                                                                                                      # 팩맨 입 열었는지 닫았는지 여부
time = 0                                                                                                                        # 게임 시작했는지 여부
playing = 1                                                                                                                     # 게임이 끝났는지 여부
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성
image2 = ["./이미지/팩맨2 왼쪽.png","./이미지/팩맨2 오른쪽.png","./이미지/팩맨2 위쪽.png","./이미지/팩맨2 아래쪽.png"]             # 팩맨 입 닫은 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN and playing == 1:                                                                       # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경
                    direct = 1                                                                                                  # 픽맨 입 연것으로 재설정
                    time = 1                                                                                                    # 게임이 시작 된 것으로 설정

    cnt += 1                                                                                                                    # FPS 시간 증가

    # 만약 게임이 시작되지 않았다면 초기 시간 초기화
    if time == 0:                                                                                                               # 시작이 안되었을 경우
        start_ticks = pygame.time.get_ticks()                                                                                   # 시작 시간 초기화
        font = pygame.font.Font(None, 50)                                                                                       # 폰트 설정
        cnt = 0                                                                                                                 # FPS 계산 초기화

    # FPS 시간 8마다 입 다물거나 열기
    for i in range(4):                                                                                                          # i는 0에서 3까지에 대해
        if [left_go,right_go,up_go,down_go][i] == True and cnt%8 == 0:                                                          # i방향으로 이동하면서 FPS 8로 나누어 떨어지면서
            if direct == 1:                                                                                                     # 팩맨이 입을 열었을 경우
                pacman.put_img(image2[i])                                                                                       # 팩맨을 입 닫은 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 2                                                                                                      # 입을 닫은 것으로 재설정
            elif direct == 2:                                                                                                   # 팩맨이 입을 닫았을 경우
                pacman.put_img(image[i])                                                                                        # 팩맨을 입 연 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 1                                                                                                      # 입을 연 것으로 재설정

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    # 지울 먹이 리스트를 만들어 팩맨과 먹이가 충돌했을 때(먹었을 때) 리스트에 추가
    remove_point_list = []                                                                                                      # 먹은 리스트 생성
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        if crash(pacman,p):                                                                                                     # 팩맨과 충돌하면
            remove_point_list.append(p)                                                                                         # 그 먹이를 먹은 리스트에 추가

    # 지울 먹이 리스트에 있는 모든 먹이들을 제거
    for dp in remove_point_list:                                                                                                # 모든 먹은 먹이에 대해
        point_list.remove(dp)                                                                                                   # 그 먹이를 먹이 리스트에서 제거
    
    # 남은 먹이들 그리기
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        p.show()                                                                                                                # 먹이 그리기
    
    # 먹이를 다 먹었을 경우 설정
    if len(point_list) == 0:                                                                                                    # 먹이 리스트의 길이가 0일 경우 (모든 먹이를 먹었을 경우)
        game_success.show()                                                                                                     # 게임 성공 이미지 그리기
        playing = 0                                                                                                             # 게임 끝난 것으로 설정
        left_go = False                                                                                                         # 이동방향 초기화
        right_go = False                                                                                                        # 이동방향 초기화
        up_go = False                                                                                                           # 이동방향 초기화
        down_go = False                                                                                                         # 이동방향 초기화
        # 게임 진행 시간 표시
        screen.blit(time_text, (400, 750))                                                                                      # 가운데에 맞추어 시간 표시
    
    # 만약 게임 중이라면 초를 다시 계산하여 표시
    if playing == 1:                                                                                                            # 게임이 끝나지 않았디면
        seconds = (pygame.time.get_ticks() - start_ticks) / 1000                                                                # 초 계산
        minute = int(seconds)//60                                                                                               # 분 계산
        seconds -= minute*60                                                                                                    # 초 재설정
        if minute == 0:                                                                                                         # 0분일 경우
            time_text = font.render(f"Time : {seconds:.2f} sec", True, (255,255,255))                                           # 초만 표시하여 폰트에 맞춰 저장
        else:                                                                                                                   # a분일 경우 (a > 0)
            time_text = font.render(f"Time : {minute:} min {seconds:.2f} sec", True, (255,255,255))                             # 분, 초를 표시하여 폰트에 맞춰 저장
        screen.blit(time_text, (10, 10))                                                                                        # 왼쪽 위에 맞추어 시간 표시

    if cnt == 88: cnt = 0                                                                                                       # FPS 시간이 너무 커지는 것을 방지하기 위해 초기화
    
    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 유령 생성

In [19]:
import pygame

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    
# 충돌 감지
def crash(a, b):
    if (a.x-b.sx <= b.x <= a.x+a.sx):                                                                                           # x의 위치가 겹치면서
        if (a.y-b.sy <= b.y <= a.y+a.sy):                                                                                       # y의 위치가 겹칠 경우
            return True                                                                                                         # 충돌했다 판정
    return False                                                                                                                # 그렇지 않을 경우 충돌하지 않았다 판정

# 게임 성공했을 때 사용할 배경 객체
game_success = obj()                                                                                                            # 객체 생성
game_success.put_img("./이미지/게임 성공.png")                                                                                  # 이미지 추가
game_success.change_size(size[0],size[1])                                                                                       # 이미지 크기 변경

# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 팩맨 먹이 리스트 생성
point_list = []
for x in range(20, size[0], 60):
    for y in range(200, size[1], 60):
        point = obj()                                                                                                           # 객체 생성
        point.put_img("./이미지/점.png")                                                                                        # 이미지 추가
        point.change_size(10,10)                                                                                                # 이미지 크기 변경
        point.x = x                                                                                                             # 객체의 초기 x위치 지정
        point.y = y                                                                                                             # 객체의 초기 y위치 지정
        point_list.append(point)                                                                                                # 먹이 리스트에 추가

# 유령 리스트 생성
monster_list = []
image3 = ["./이미지/분홍 유령.png", "./이미지/빨강 유령.png", "./이미지/파랑 유령.png"]                                            # 유령 이미지 리스트 생성
for x in range(35,size[0],301):
    monster = obj()                                                                                                             # 객체 생성
    monster.put_img(image3[x%3])                                                                                                # 이미지 추가
    monster.change_size(50, 50)                                                                                                 # 이미지 크기 변경
    monster.x = x                                                                                                               # 객체의 초기 x위치 지정
    monster.y = 10                                                                                                              # 객체의 초기 y위치 지정
    monster_list.append(monster)                                                                                                # 몬스터 리스트에 추가

# 필요한 변수 선언
cnt = 0                                                                                                                         # FPS 시간 계산
direct = 1                                                                                                                      # 팩맨 입 열었는지 닫았는지 여부
time = 0                                                                                                                        # 게임 시작했는지 여부
playing = 1                                                                                                                     # 게임이 끝났는지 여부
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성
image2 = ["./이미지/팩맨2 왼쪽.png","./이미지/팩맨2 오른쪽.png","./이미지/팩맨2 위쪽.png","./이미지/팩맨2 아래쪽.png"]             # 팩맨 입 닫은 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN and playing == 1:                                                                       # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경
                    direct = 1                                                                                                  # 픽맨 입 연것으로 재설정
                    time = 1                                                                                                    # 게임이 시작 된 것으로 설정

    cnt += 1                                                                                                                    # FPS 시간 증가

    # 만약 게임이 시작되지 않았다면 초기 시간 초기화
    if time == 0:                                                                                                               # 시작이 안되었을 경우
        start_ticks = pygame.time.get_ticks()                                                                                   # 시작 시간 초기화
        font = pygame.font.Font(None, 50)                                                                                       # 폰트 설정
        cnt = 0                                                                                                                 # FPS 계산 초기화

    # FPS 시간 8마다 입 다물거나 열기
    for i in range(4):                                                                                                          # i는 0에서 3까지에 대해
        if [left_go,right_go,up_go,down_go][i] == True and cnt%8 == 0:                                                          # i방향으로 이동하면서 FPS 8로 나누어 떨어지면서
            if direct == 1:                                                                                                     # 팩맨이 입을 열었을 경우
                pacman.put_img(image2[i])                                                                                       # 팩맨을 입 닫은 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 2                                                                                                      # 입을 닫은 것으로 재설정
            elif direct == 2:                                                                                                   # 팩맨이 입을 닫았을 경우
                pacman.put_img(image[i])                                                                                        # 팩맨을 입 연 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 1                                                                                                      # 입을 연 것으로 재설정

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    # 지울 먹이 리스트를 만들어 팩맨과 먹이가 충돌했을 때(먹었을 때) 리스트에 추가
    remove_point_list = []                                                                                                      # 먹은 리스트 생성
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        if crash(pacman,p):                                                                                                     # 팩맨과 충돌하면
            remove_point_list.append(p)                                                                                         # 그 먹이를 먹은 리스트에 추가

    # 지울 먹이 리스트에 있는 모든 먹이들을 제거
    for dp in remove_point_list:                                                                                                # 모든 먹은 먹이에 대해
        point_list.remove(dp)                                                                                                   # 그 먹이를 먹이 리스트에서 제거
    
    # 남은 먹이들 그리기
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        p.show()                                                                                                                # 먹이 그리기
    
    # 유령들 그리기
    for m in monster_list:                                                                                                      # 모든 몬스터에 대해
        m.show()                                                                                                                # 몬스터 그리기

    # 먹이를 다 먹었을 경우 설정
    if len(point_list) == 0:                                                                                                    # 먹이 리스트의 길이가 0일 경우 (모든 먹이를 먹었을 경우)
        game_success.show()                                                                                                     # 게임 성공 이미지 그리기
        playing = 0                                                                                                             # 게임 끝난 것으로 설정
        left_go = False                                                                                                         # 이동방향 초기화
        right_go = False                                                                                                        # 이동방향 초기화
        up_go = False                                                                                                           # 이동방향 초기화
        down_go = False                                                                                                         # 이동방향 초기화
        # 게임 진행 시간 표시
        screen.blit(time_text, (400, 750))                                                                                      # 가운데에 맞추어 시간 표시
    
    # 만약 게임 중이라면 초를 다시 계산하여 표시
    if playing == 1:                                                                                                            # 게임이 끝나지 않았디면
        seconds = (pygame.time.get_ticks() - start_ticks) / 1000                                                                # 초 계산
        minute = int(seconds)//60                                                                                               # 분 계산
        seconds -= minute*60                                                                                                    # 초 재설정
        if minute == 0:                                                                                                         # 0분일 경우
            time_text = font.render(f"Time : {seconds:.2f} sec", True, (255,255,255))                                           # 초만 표시하여 폰트에 맞춰 저장
        else:                                                                                                                   # a분일 경우 (a > 0)
            time_text = font.render(f"Time : {minute:} min {seconds:.2f} sec", True, (255,255,255))                             # 분, 초를 표시하여 폰트에 맞춰 저장
        screen.blit(time_text, (10, 10))                                                                                        # 왼쪽 위에 맞추어 시간 표시

    if cnt == 88: cnt = 0                                                                                                       # FPS 시간이 너무 커지는 것을 방지하기 위해 초기화
    
    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 유령 이동

In [20]:
import pygame
import random
import math

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    # 객체 이동 방향 설정
    def set_move(self, x, y):
        self.move_x = x                                                                                                         # x방향으로 이동할 크기
        self.move_y = y                                                                                                         # y방향으로 이동할 크기
    # 객체 이동방향대로 위치 이동
    def update(self, width, height):
        if self.x < 0 or self.x + self.sx > width:                                                                              # 객체의 x위치가 화면을 벗어날 경우
            self.move_x = -self.move_x                                                                                          # 반대방향으로 이동방향 바꾸기
        if self.y < 0 or self.y + self.sy > height:                                                                             # 객체의 y위치가 화면을 벗어날 경우
            self.move_y = -self.move_y                                                                                          # 반대방향으로 이동방향 바꾸기
        self.x += self.move_x                                                                                                   # 객체의 x위치 이동
        self.y += self.move_y                                                                                                   # 객체의 y위치 이동

# 충돌 감지
def crash(a, b):
    if (a.x-b.sx <= b.x <= a.x+a.sx):                                                                                           # x의 위치가 겹치면서
        if (a.y-b.sy <= b.y <= a.y+a.sy):                                                                                       # y의 위치가 겹칠 경우
            return True                                                                                                         # 충돌했다 판정
    return False                                                                                                                # 그렇지 않을 경우 충돌하지 않았다 판정

# 게임 성공했을 때 사용할 배경 객체
game_success = obj()                                                                                                            # 객체 생성
game_success.put_img("./이미지/게임 성공.png")                                                                                  # 이미지 추가
game_success.change_size(size[0],size[1])                                                                                       # 이미지 크기 변경

# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 팩맨 먹이 리스트 생성
point_list = []
for x in range(20, size[0], 60):
    for y in range(200, size[1], 60):
        point = obj()                                                                                                           # 객체 생성
        point.put_img("./이미지/점.png")                                                                                        # 이미지 추가
        point.change_size(10,10)                                                                                                # 이미지 크기 변경
        point.x = x                                                                                                             # 객체의 초기 x위치 지정
        point.y = y                                                                                                             # 객체의 초기 y위치 지정
        point_list.append(point)                                                                                                # 먹이 리스트에 추가

# 유령 리스트 생성
monster_list = []
r = 6                                                                                                                           # 객체 이동 속도 지정
image3 = ["./이미지/분홍 유령.png", "./이미지/빨강 유령.png", "./이미지/파랑 유령.png"]                                            # 유령 이미지 리스트 생성
for x in range(35,size[0],301):
    monster = obj()                                                                                                             # 객체 생성
    monster.put_img(image3[x%3])                                                                                                # 이미지 추가
    monster.change_size(50, 50)                                                                                                 # 이미지 크기 변경
    monster.x = x                                                                                                               # 객체의 초기 x위치 지정
    monster.y = 10                                                                                                              # 객체의 초기 y위치 지정
    theta = random.randint(0,360)                                                                                               # 랜덤하게 각도 지정
    monster.set_move(r*math.cos(theta), r*math.sin(theta))                                                                      # 이동속도는 r만큼 각도 방향으로 이동할 값 지정
    monster_list.append(monster)                                                                                                # 몬스터 리스트에 추가

# 필요한 변수 선언
cnt = 0                                                                                                                         # FPS 시간 계산
direct = 1                                                                                                                      # 팩맨 입 열었는지 닫았는지 여부
time = 0                                                                                                                        # 게임 시작했는지 여부
playing = 1                                                                                                                     # 게임이 끝났는지 여부
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성
image2 = ["./이미지/팩맨2 왼쪽.png","./이미지/팩맨2 오른쪽.png","./이미지/팩맨2 위쪽.png","./이미지/팩맨2 아래쪽.png"]             # 팩맨 입 닫은 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN and playing == 1:                                                                       # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경
                    direct = 1                                                                                                  # 픽맨 입 연것으로 재설정
                    time = 1                                                                                                    # 게임이 시작 된 것으로 설정

    cnt += 1                                                                                                                    # FPS 시간 증가

    # 만약 게임이 시작되지 않았다면 초기 시간 초기화
    if time == 0:                                                                                                               # 시작이 안되었을 경우
        start_ticks = pygame.time.get_ticks()                                                                                   # 시작 시간 초기화
        font = pygame.font.Font(None, 50)                                                                                       # 폰트 설정
        cnt = 0                                                                                                                 # FPS 계산 초기화

    # FPS 시간 8마다 입 다물거나 열기
    for i in range(4):                                                                                                          # i는 0에서 3까지에 대해
        if [left_go,right_go,up_go,down_go][i] == True and cnt%8 == 0:                                                          # i방향으로 이동하면서 FPS 8로 나누어 떨어지면서
            if direct == 1:                                                                                                     # 팩맨이 입을 열었을 경우
                pacman.put_img(image2[i])                                                                                       # 팩맨을 입 닫은 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 2                                                                                                      # 입을 닫은 것으로 재설정
            elif direct == 2:                                                                                                   # 팩맨이 입을 닫았을 경우
                pacman.put_img(image[i])                                                                                        # 팩맨을 입 연 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 1                                                                                                      # 입을 연 것으로 재설정

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    # 지울 먹이 리스트를 만들어 팩맨과 먹이가 충돌했을 때(먹었을 때) 리스트에 추가
    remove_point_list = []                                                                                                      # 먹은 리스트 생성
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        if crash(pacman,p):                                                                                                     # 팩맨과 충돌하면
            remove_point_list.append(p)                                                                                         # 그 먹이를 먹은 리스트에 추가

    # 지울 먹이 리스트에 있는 모든 먹이들을 제거
    for dp in remove_point_list:                                                                                                # 모든 먹은 먹이에 대해
        point_list.remove(dp)                                                                                                   # 그 먹이를 먹이 리스트에서 제거
    
    # 남은 먹이들 그리기
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        p.show()                                                                                                                # 먹이 그리기
    
    # 유령들 그리기
    for m in monster_list:                                                                                                      # 모든 몬스터에 대해
        if cnt == 88:                                                                                                           # FPS 시간이 88로 나누어 떨어질 경우
            theta = random.randint(0,360)                                                                                       # 랜덤하게 각도 지정
            m.set_move(r*math.cos(theta), r*math.sin(theta))                                                                    # 이동속도는 r만큼 각도 방향으로 이동할 값 지정
        if time == 1 and playing == 1:                                                                                          # 게임이 시작되고 끝나지 않았을 경우
            m.update(screen_width, screen_height)                                                                               # 이동 방향대로 위치 이동
        m.show()                                                                                                                # 몬스터 그리기

    # 먹이를 다 먹었을 경우 설정
    if len(point_list) == 0:                                                                                                    # 먹이 리스트의 길이가 0일 경우 (모든 먹이를 먹었을 경우)
        game_success.show()                                                                                                     # 게임 성공 이미지 그리기
        playing = 0                                                                                                             # 게임 끝난 것으로 설정
        left_go = False                                                                                                         # 이동방향 초기화
        right_go = False                                                                                                        # 이동방향 초기화
        up_go = False                                                                                                           # 이동방향 초기화
        down_go = False                                                                                                         # 이동방향 초기화
        # 게임 진행 시간 표시
        screen.blit(time_text, (400, 750))                                                                                      # 가운데에 맞추어 시간 표시
    
    # 만약 게임 중이라면 초를 다시 계산하여 표시
    if playing == 1:                                                                                                            # 게임이 끝나지 않았디면
        seconds = (pygame.time.get_ticks() - start_ticks) / 1000                                                                # 초 계산
        minute = int(seconds)//60                                                                                               # 분 계산
        seconds -= minute*60                                                                                                    # 초 재설정
        if minute == 0:                                                                                                         # 0분일 경우
            time_text = font.render(f"Time : {seconds:.2f} sec", True, (255,255,255))                                           # 초만 표시하여 폰트에 맞춰 저장
        else:                                                                                                                   # a분일 경우 (a > 0)
            time_text = font.render(f"Time : {minute:} min {seconds:.2f} sec", True, (255,255,255))                             # 분, 초를 표시하여 폰트에 맞춰 저장
        screen.blit(time_text, (10, 10))                                                                                        # 왼쪽 위에 맞추어 시간 표시

    if cnt == 88: cnt = 0                                                                                                       # FPS 시간이 너무 커지는 것을 방지하기 위해 초기화
    
    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료

# 팩맨과 유령 충돌 감지 (마지막)

In [21]:
import pygame
import random
import math

# Pygame 초기화
pygame.init()

# 게임 화면 설정
screen_width = 1000                                                                                                             # 화면 가로 설정
screen_height = 1000                                                                                                            # 화면 세로 설정
size = [screen_width,screen_height]                                                                                             # 화면 가로, 세로 합치기
screen = pygame.display.set_mode(size)                                                                                          # 화면 만들기

# 제목 설정
title = 'PAC MAN'                                                                                                               # 게임 제목 설정
pygame.display.set_caption(title)                                                                                               # 게임 제목 쓰기

# 게임 내 설정
clock = pygame.time.Clock()                                                                                                     # 게임 내 시간 설정
color = (120,120,255)                                                                                                           # 화면에 그릴 색상 설정

# 객체 생성
class obj:
    # 초기 위치 설정
    def __init__(self):
        self.x = 0
        self.y = 0
    # 이미지 추가
    def put_img(self, address):
        self.img = pygame.image.load(address).convert_alpha()                                                                   # 이미지 주소로 부터 이미지 추가
        self.sx, self.sy = self.img.get_size()                                                                                  # 이미지로 부터 사이즈 받아오기
    # 이미지 사이즈 변경
    def change_size(self, sx, sy):
        self.img = pygame.transform.scale(self.img,(sx,sy))                                                                     # 이미지의 사이즈를 재설정
        self.sx, self.sy = self.img.get_size()                                                                                  # 재설정 후 이미지로 부터 사이즈 받아오기
    # 객체 그리기
    def show(self):
        screen.blit(self.img, (self.x, self.y))                                                                                 # 화면에 위치에 맞게 그려 넣기
    # 객체 이동 방향 설정
    def set_move(self, x, y):
        self.move_x = x                                                                                                         # x방향으로 이동할 크기
        self.move_y = y                                                                                                         # y방향으로 이동할 크기
    # 객체 이동방향대로 위치 이동
    def update(self, width, height):
        if self.x < 0 or self.x + self.sx > width:                                                                              # 객체의 x위치가 화면을 벗어날 경우
            self.move_x = -self.move_x                                                                                          # 반대방향으로 이동방향 바꾸기
        if self.y < 0 or self.y + self.sy > height:                                                                             # 객체의 y위치가 화면을 벗어날 경우
            self.move_y = -self.move_y                                                                                          # 반대방향으로 이동방향 바꾸기
        self.x += self.move_x                                                                                                   # 객체의 x위치 이동
        self.y += self.move_y                                                                                                   # 객체의 y위치 이동

# 충돌 감지
def crash(a, b):
    if (a.x-b.sx <= b.x <= a.x+a.sx):                                                                                           # x의 위치가 겹치면서
        if (a.y-b.sy <= b.y <= a.y+a.sy):                                                                                       # y의 위치가 겹칠 경우
            return True                                                                                                         # 충돌했다 판정
    return False                                                                                                                # 그렇지 않을 경우 충돌하지 않았다 판정

# 게임 실패했을 때 사용할 배경 객체
game_over = obj()                                                                                                               # 객체 생성
game_over.put_img("./이미지/게임 오버.png")                                                                                     # 이미지 추가
game_over.change_size(size[0],size[1])                                                                                          # 이미지 크기 변경

# 게임 성공했을 때 사용할 배경 객체
game_success = obj()                                                                                                            # 객체 생성
game_success.put_img("./이미지/게임 성공.png")                                                                                  # 이미지 추가
game_success.change_size(size[0],size[1])                                                                                       # 이미지 크기 변경

# 팩맨 객체 생성
pacman = obj()                                                                                                                  # 객체 생성
pacman.put_img("./이미지/팩맨1 위쪽.png")                                                                                       # 이미지 추가
pacman.change_size(40,40)                                                                                                       # 이미지 크기 변경
pacman.x = 480                                                                                                                  # 객체의 초기 x위치 지정
pacman.y = 600                                                                                                                  # 객체의 초기 y위치 지정
pacman.move = 8                                                                                                                 # 객체의 이동 속도 지정

# 팩맨 객체 이동방향 초기 값
left_go = False
right_go = False
up_go = False
down_go = False

# 팩맨 먹이 리스트 생성
point_list = []
for x in range(20, size[0], 60):
    for y in range(200, size[1], 60):
        point = obj()                                                                                                           # 객체 생성
        point.put_img("./이미지/점.png")                                                                                        # 이미지 추가
        point.change_size(10,10)                                                                                                # 이미지 크기 변경
        point.x = x                                                                                                             # 객체의 초기 x위치 지정
        point.y = y                                                                                                             # 객체의 초기 y위치 지정
        point_list.append(point)                                                                                                # 먹이 리스트에 추가

# 유령 리스트 생성
monster_list = []
r = 6                                                                                                                           # 객체 이동 속도 지정
image3 = ["./이미지/분홍 유령.png", "./이미지/빨강 유령.png", "./이미지/파랑 유령.png"]                                            # 유령 이미지 리스트 생성
for x in range(35,size[0],301):
    monster = obj()                                                                                                             # 객체 생성
    monster.put_img(image3[x%3])                                                                                                # 이미지 추가
    monster.change_size(50, 50)                                                                                                 # 이미지 크기 변경
    monster.x = x                                                                                                               # 객체의 초기 x위치 지정
    monster.y = 10                                                                                                              # 객체의 초기 y위치 지정
    theta = random.randint(0,360)                                                                                               # 랜덤하게 각도 지정
    monster.set_move(r*math.cos(theta), r*math.sin(theta))                                                                      # 이동속도는 r만큼 각도 방향으로 이동할 값 지정
    monster_list.append(monster)                                                                                                # 몬스터 리스트에 추가

# 필요한 변수 선언
cnt = 0                                                                                                                         # FPS 시간 계산
direct = 1                                                                                                                      # 팩맨 입 열었는지 닫았는지 여부
time = 0                                                                                                                        # 게임 시작했는지 여부
playing = 1                                                                                                                     # 게임이 끝났는지 여부
key = [pygame.K_LEFT,pygame.K_RIGHT,pygame.K_UP,pygame.K_DOWN]                                                                  # 화살표 키입력 리스트 생성
image = ["./이미지/팩맨1 왼쪽.png","./이미지/팩맨1 오른쪽.png","./이미지/팩맨1 위쪽.png","./이미지/팩맨1 아래쪽.png"]              # 팩맨 입 연 이미지 리스트 생성
image2 = ["./이미지/팩맨2 왼쪽.png","./이미지/팩맨2 오른쪽.png","./이미지/팩맨2 위쪽.png","./이미지/팩맨2 아래쪽.png"]             # 팩맨 입 닫은 이미지 리스트 생성

# 게임 루프
running = True                                                                                                                  # 창이 실행중 여부
while running:                                                                                                                  # 창이 실행중이라면

    clock.tick(60)                                                                                                              # FPS 설정

    # event가 일어났는지 확인
    for event in pygame.event.get():                                                                                            # event가 일어났는지 확인
        # 창 X표시 누르면 게임 종료
        if event.type == pygame.QUIT:                                                                                           # 창 X표시 누를 경우
            running = False                                                                                                     # 게임 종료
        # 키 입력 확인
        if event.type == pygame.KEYDOWN and playing == 1:                                                                       # 키가 눌렸고 게임이 끝나지 않았을 경우
            for i in range(4):                                                                                                  # i는 0에서 3까지에 대해
                if event.key == key[i]:                                                                                         # key의 i번째 키가 눌리면
                    go = [1 if i == j else 0 for j in range(4)]                                                                 # i방향을 제외하고 모두 0으로 설정
                    left_go = bool(go[0])                                                                                       # bool 변환
                    right_go = bool(go[1])
                    up_go = bool(go[2])
                    down_go = bool(go[3])
                    pacman.put_img(image[i])                                                                                    # 이미지 재설정
                    pacman.change_size(40,40)                                                                                   # 이미지 크기 변경
                    direct = 1                                                                                                  # 픽맨 입 연것으로 재설정
                    time = 1                                                                                                    # 게임이 시작 된 것으로 설정

    cnt += 1                                                                                                                    # FPS 시간 증가

    # 만약 게임이 시작되지 않았다면 초기 시간 초기화
    if time == 0:                                                                                                               # 시작이 안되었을 경우
        start_ticks = pygame.time.get_ticks()                                                                                   # 시작 시간 초기화
        font = pygame.font.Font(None, 50)                                                                                       # 폰트 설정
        cnt = 0                                                                                                                 # FPS 계산 초기화

    # FPS 시간 8마다 입 다물거나 열기
    for i in range(4):                                                                                                          # i는 0에서 3까지에 대해
        if [left_go,right_go,up_go,down_go][i] == True and cnt%8 == 0:                                                          # i방향으로 이동하면서 FPS 8로 나누어 떨어지면서
            if direct == 1:                                                                                                     # 팩맨이 입을 열었을 경우
                pacman.put_img(image2[i])                                                                                       # 팩맨을 입 닫은 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 2                                                                                                      # 입을 닫은 것으로 재설정
            elif direct == 2:                                                                                                   # 팩맨이 입을 닫았을 경우
                pacman.put_img(image[i])                                                                                        # 팩맨을 입 연 이미지로 재설정
                pacman.change_size(40,40)                                                                                       # 이미지 크기 변경
                direct = 1                                                                                                      # 입을 연 것으로 재설정

    # 각 이동 방향으로 이동
    if left_go == True:                                                                                                         # 왼쪽으로 이동할 경우
        pacman.x -= pacman.move                                                                                                 # 이동속도만큼 x값을 빼준다
        if pacman.x <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.x = 0                                                                                                        # 끝으로 초기화
    if right_go == True:                                                                                                        # 오른쪽으로 이동할 경우
        pacman.x += pacman.move                                                                                                 # 이동속도만큼 x값을 더해준다
        if pacman.x >= size[0] - pacman.sx:                                                                                     # 화면밖으로 나갈 경우
            pacman.x = size[0] - pacman.sx                                                                                      # 끝으로 초기화
    if up_go == True:                                                                                                           # 위쪽으로 이동할 경우
        pacman.y -= pacman.move                                                                                                 # 이동속도만큼 y값을 빼준다
        if pacman.y <= 0:                                                                                                       # 화면밖으로 나갈 경우
            pacman.y = 0                                                                                                        # 끝으로 초기화
    if down_go == True:                                                                                                         # 아래쪽으로 이동할 경우
        pacman.y += pacman.move                                                                                                 # 이동속도만큼 y값을 더해준다
        if pacman.y >= size[1] - pacman.sy:                                                                                     # 화면밖으로 나갈 경우
            pacman.y = size[1] - pacman.sy                                                                                      # 끝으로 초기화

    screen.fill(color)                                                                                                          # 화면 채우기

    pacman.show()                                                                                                               # 팩맨 그리기

    # 지울 먹이 리스트를 만들어 팩맨과 먹이가 충돌했을 때(먹었을 때) 리스트에 추가
    remove_point_list = []                                                                                                      # 먹은 리스트 생성
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        if crash(pacman,p):                                                                                                     # 팩맨과 충돌하면
            remove_point_list.append(p)                                                                                         # 그 먹이를 먹은 리스트에 추가

    # 지울 먹이 리스트에 있는 모든 먹이들을 제거
    for dp in remove_point_list:                                                                                                # 모든 먹은 먹이에 대해
        point_list.remove(dp)                                                                                                   # 그 먹이를 먹이 리스트에서 제거
    
    # 남은 먹이들 그리기
    for p in point_list:                                                                                                        # 모든 먹이에 대해
        p.show()                                                                                                                # 먹이 그리기
    
    # 유령들 그리기
    for m in monster_list:                                                                                                      # 모든 몬스터에 대해
        if cnt == 88:                                                                                                           # FPS 시간이 88로 나누어 떨어질 경우
            theta = random.randint(0,360)                                                                                       # 랜덤하게 각도 지정
            m.set_move(r*math.cos(theta), r*math.sin(theta))                                                                    # 이동속도는 r만큼 각도 방향으로 이동할 값 지정
        if time == 1 and playing == 1:                                                                                          # 게임이 시작되고 끝나지 않았을 경우
            m.update(screen_width, screen_height)                                                                               # 이동 방향대로 위치 이동
        m.show()                                                                                                                # 몬스터 그리기
        if crash(pacman,m):                                                                                                     # 팩맨과 유령이 충돌했을 경우
            game_over.show()                                                                                                    # 게임 오버 이미지 그리기
            playing = 0                                                                                                         # 게임 끝난 것으로 설정
            left_go = False                                                                                                     # 이동방향 초기화
            right_go = False                                                                                                    # 이동방향 초기화
            up_go = False                                                                                                       # 이동방향 초기화
            down_go = False                                                                                                     # 이동방향 초기화
            # 게임 진행 시간 표시
            screen.blit(time_text, (400, 750))                                                                                  # 가운데에 맞추어 시간 표시
            break                                                                                                               # for문 벗어나기
    
    # 먹이를 다 먹었을 경우 설정
    if len(point_list) == 0:                                                                                                    # 먹이 리스트의 길이가 0일 경우 (모든 먹이를 먹었을 경우)
        game_success.show()                                                                                                     # 게임 성공 이미지 그리기
        playing = 0                                                                                                             # 게임 끝난 것으로 설정
        left_go = False                                                                                                         # 이동방향 초기화
        right_go = False                                                                                                        # 이동방향 초기화
        up_go = False                                                                                                           # 이동방향 초기화
        down_go = False                                                                                                         # 이동방향 초기화
        # 게임 진행 시간 표시
        screen.blit(time_text, (400, 750))                                                                                      # 가운데에 맞추어 시간 표시
    
    # 만약 게임 중이라면 초를 다시 계산하여 표시
    if playing == 1:                                                                                                            # 게임이 끝나지 않았디면
        seconds = (pygame.time.get_ticks() - start_ticks) / 1000                                                                # 초 계산
        minute = int(seconds)//60                                                                                               # 분 계산
        seconds -= minute*60                                                                                                    # 초 재설정
        if minute == 0:                                                                                                         # 0분일 경우
            time_text = font.render(f"Time : {seconds:.2f} sec", True, (255,255,255))                                           # 초만 표시하여 폰트에 맞춰 저장
        else:                                                                                                                   # a분일 경우 (a > 0)
            time_text = font.render(f"Time : {minute:} min {seconds:.2f} sec", True, (255,255,255))                             # 분, 초를 표시하여 폰트에 맞춰 저장
        screen.blit(time_text, (10, 10))                                                                                        # 왼쪽 위에 맞추어 시간 표시

    if cnt == 88: cnt = 0                                                                                                       # FPS 시간이 너무 커지는 것을 방지하기 위해 초기화
    
    pygame.display.flip()                                                                                                       # 화면을 매 프레임마다 업데이트

pygame.quit()                                                                                                                   # pygame 종료